In [1]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string

Using TensorFlow backend.


In [2]:
data = pd.read_csv('Resources/LabelledData.csv', header = None, delimiter='\t')

In [3]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets
    """
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [4]:
import csv
training = []
line_counter = 0
with open("Resources/LabelledData.csv",'r') as f:
    tsv = csv.reader(f, dialect='excel', delimiter='\t')
    for line in tsv:
        if line_counter < 900:
            training.append(line[0])
        else:
            break
            line_counter = line_counter+1
    # training_data = list(training)
    training_data = [s.strip() for s in training]
    # Split by words
    x_text = training_data
    x_text = [clean_str(sent) for sent in x_text]  

In [5]:
t = open('check.txt','wt')
label =[]
labels=[]
line_counter = 0
with open("Resources/LabelledData.csv",'r') as f:
    tsv = csv.reader(f, dialect='excel', delimiter='\t')
    for line in tsv:
        if line_counter<900:
            label.append(line[1])
        else:
            break
            line_counter =  line_counter +1
    labels =label
    tempList = []
    labelList = []
    for a in label:
        t.write(a)
        t.write('--')
        if not a in tempList:
            tempList.append(a)
            labelList.append(tempList.index(a))
            t.write(str(tempList.index(a)))
            t.write('\n')
        else:
            labelList.append(tempList.index(a))
    n_labels = len(labelList)
    target = []
    for x in range(0,n_labels):
        listFori = []
        i=0
        while i < len(tempList):
            if i == labelList[x]:
                listFori.append(1)
            else:
                listFori.append(0)
            i = i+1
        target.append(listFori)
    y = np.concatenate([target],0)

In [6]:
MAX_WORDS = 10000
MAX_SEQUENCE_LENGTH = 1000
EMBEDDING_DIM = 100
# filter sizes of the different conv layers 
filter_sizes = [3,4,5]
num_filters = 512
embedding_dim = 100
# dropout probability
drop = 0.5
batch_size = 30
epochs =30

In [7]:
GLOVE_DIR='./GLOVE.6B/'
embeddings_index = {}
FILE=os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')
f = open(FILE,encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
tokenizer  = Tokenizer(num_words = MAX_WORDS)
print(tokenizer.fit_on_texts(x_text))
sequences =  tokenizer.texts_to_sequences(x_text)
print(len(sequences))
word_index = tokenizer.word_index
print("unique words : {}".format(len(word_index)))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(y))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', y.shape)
print(len(labels))

None
1483
unique words : 3674
Shape of data tensor: (1483, 1000)
Shape of label tensor: (1483, 5)
1483


In [9]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
y = y[indices]
nb_validation_samples = int(0.10 * data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = y[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = y[-nb_validation_samples:]

In [10]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [11]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                               input_length=MAX_SEQUENCE_LENGTH,
                             trainable=False)

In [12]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from keras.optimizers import Adam

from keras.utils import plot_model

In [13]:
model = Sequential()

# Embedding layer
model.add(Embedding(input_dim=len(word_index) + 1,output_dim=100,weights=None,trainable=True))

# Recurrent layer
model.add( LSTM(64, return_sequences=False, dropout=0.1,recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(units=5, activation='softmax'))
checkpoint = ModelCheckpoint('weights_cnn_sentece.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 100)         367500    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
Total params: 414,225
Trainable params: 414,225
Non-trainable params: 0
_________________________________________________________________


In [14]:
print("Traning Model...")
model.fit(x_train, y_train, batch_size=20, epochs=15, verbose=1, callbacks=[checkpoint])

Traning Model...


C:\Users\Chindukuri\.conda\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
1335/1335 [==============================] - 108s 81ms/step - loss: 1.4463 - accuracy: 0.4060
Epoch 2/15


C:\Users\Chindukuri\.conda\envs\tensorflow\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


1335/1335 [==============================] - 107s 80ms/step - loss: 1.1097 - accuracy: 0.5543
Epoch 3/15
1335/1335 [==============================] - 107s 81ms/step - loss: 0.5226 - accuracy: 0.8307
Epoch 4/15
1335/1335 [==============================] - 111s 83ms/step - loss: 0.3374 - accuracy: 0.8509
Epoch 5/15
1335/1335 [==============================] - 115s 86ms/step - loss: 0.2518 - accuracy: 0.8966
Epoch 6/15
1335/1335 [==============================] - 117s 88ms/step - loss: 0.1698 - accuracy: 0.9468
Epoch 7/15
1335/1335 [==============================] - 124s 93ms/step - loss: 0.1010 - accuracy: 0.9760
Epoch 8/15
1335/1335 [==============================] - 130s 98ms/step - loss: 0.0370 - accuracy: 0.9948
Epoch 9/15
1335/1335 [==============================] - 138s 103ms/step - loss: 0.0234 - accuracy: 0.9970
Epoch 10/15
1335/1335 [==============================] - 143s 107ms/step - loss: 0.0187 - accuracy: 0.9955
Epoch 11/15
1335/1335 [==============================] - 147s 1

In [15]:
score = model.evaluate(x_test, y_test, verbose=0)  
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.3508598127880612
Test accuracy: 0.9527027010917664


In [16]:
predictions = model.predict(x_test, batch_size=1024,   verbose=1)
labels = ['unknown', 'what', 'when', 'who', 'affirmation']
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

148/148 [==============================] - 4s 29ms/step


In [17]:
prediction_labels

['who',
 'affirmation',
 'who',
 'affirmation',
 'what',
 'who',
 'affirmation',
 'who',
 'unknown',
 'who',
 'who',
 'what',
 'who',
 'unknown',
 'who',
 'who',
 'when',
 'when',
 'unknown',
 'who',
 'who',
 'who',
 'what',
 'what',
 'unknown',
 'who',
 'what',
 'unknown',
 'unknown',
 'what',
 'what',
 'when',
 'what',
 'what',
 'who',
 'who',
 'who',
 'who',
 'who',
 'what',
 'when',
 'who',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'what',
 'who',
 'who',
 'who',
 'who',
 'unknown',
 'what',
 'what',
 'what',
 'unknown',
 'who',
 'what',
 'who',
 'what',
 'when',
 'unknown',
 'who',
 'who',
 'affirmation',
 'what',
 'who',
 'when',
 'who',
 'unknown',
 'who',
 'when',
 'who',
 'unknown',
 'who',
 'who',
 'who',
 'what',
 'who',
 'what',
 'when',
 'unknown',
 'who',
 'what',
 'who',
 'what',
 'who',
 'what',
 'unknown',
 'what',
 'affirmation',
 'unknown',
 'what',
 'who',
 'what',
 'what',
 'who',
 'who',
 'what',
 'who',
 'what',
 'unknown',
 'who',
 'who',
 'what',
 'who',

In [30]:
 q=[
'what fowl grabs the spotlight after the chinese year of the monkey',
'what is the full form of .com',
'what contemptible scoundrel stole the cork from my lunch',
'what team did baseball s st. louis browns become',
'what is the oldest profession',
'what are liver enzymes',
'name the scar-faced bounty hunter of the old west', 
'when was ozzy osbourne born',
'why do heavier objects travel downhill faster',
'who was the pride of the yankees',
'who killed gandhi',
'what is considered the costliest disaster the insurance industry has ever faced',
'what sprawling u.s. state boasts the most airports',
'what did the only repealed amendment to the u.s. constitution deal with',
'how many jews were executed in concentration camps during wwii',
'what is nine inch nails',
'what is an annotated bibliography',
'what is the date of boxing day',
'what articles of clothing are tokens in monopoly',
'name 11 famous martyrs',
'what is the olympic motto']

In [31]:
tokenizer  = Tokenizer(num_words = MAX_WORDS)
print(tokenizer.fit_on_texts(q))
sequences =  tokenizer.texts_to_sequences(q)
print(len(sequences))
word_index = tokenizer.word_index
print("unique words : {}".format(len(word_index)))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

None
21
unique words : 106


In [32]:
predictions = model.predict(data, batch_size=1024,   verbose=1)
labels = ['unknown', 'what', 'when', 'who', 'affirmation']
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

21/21 [==============================] - 0s 11ms/step


In [33]:
prediction_labels

['affirmation',
 'what',
 'unknown',
 'what',
 'what',
 'what',
 'who',
 'when',
 'unknown',
 'unknown',
 'unknown',
 'who',
 'what',
 'what',
 'who',
 'what',
 'who',
 'what',
 'what',
 'unknown',
 'what']